In [1]:
import pathlib
import re
import numpy as np

NAMETABLES = pathlib.Path("gfx/nametables") 

OG_GAME_NAMETABLE =  NAMETABLES / "game_nametable_og.bin"
GAME_NAMETABLE =  NAMETABLES / "game_nametable.bin"

OG_TITLE_NAMETABLE = NAMETABLES /  "title_screen_nametable_og.bin"
TITLE_NAMETABLE =  NAMETABLES / "title_screen_nametable.bin"

LEVEL_NAMETABLE =  NAMETABLES / "level_menu_nametable.bin"
OG_LEVEL_NAMETABLE =  NAMETABLES / "level_menu_nametable_og.bin"

In [2]:

def nt_to_raw(nametable):
    results = []
    with open(nametable, "rb") as file:
        while True:
            chunk = file.read(3)
            if not chunk or chunk == 0xFF:
                break
            length = chunk[-1]
            # print (chunk[0],chunk[1])
            results.extend(list(file.read(length)))
    return results


In [3]:
def raw_to_nt(ntbytes, filename):
    start_byte = 0x2000
    suffix = 0x20
    finale = 0xFF
    results = []
    ntbytes_len = len(ntbytes)
    # print(f'{ntbytes_len}')
    for i in range(0, ntbytes_len, 32):
        chunk = ntbytes[i: i + 32]
        results.extend(list((start_byte + i).to_bytes(2, byteorder="big")))
        results.append(suffix)
        results.extend(chunk)
    results.append(finale)
    with open(filename, 'wb') as file:
        file.write(bytes(results) )



In [4]:
def get_array_grid_and_footer(path):

    ntbytes = nt_to_raw(path)
    attr_array = np.array(ntbytes, dtype=np.uint8)
    footer: np.array = attr_array[-64:]
    footer.shape = 8,8

    nametable = attr_array[:-64]

    nametable_grid = nametable.copy()
    nametable_grid.shape = 30,32
    return nametable, nametable_grid, footer


In [5]:
def print_attr_table(footer):
    print("Original Attr Table:")

    for i, row in enumerate(footer):
        line1 = ""
        line2 = ""
        for byte in row:
            tl =  byte & 0b00000011
            tr = (byte & 0b00001100) >> 2
            bl = (byte & 0b00110000) >> 4
            br = (byte & 0b11000000) >> 6
            line1 += f'{tl}{tr}'
            line2 += f'{bl}{br}'
        print(line1, sep='')
        print(line2, sep='')

In [6]:
import string

def print_and_get_filler(nametable):
    filler = "abcdefghijklmnopqrstuvwxyz!#$%&'()*+,-/:;<=>?@[]^_`{|}~¹º»¼½¾¿ÀÁÂÃÄÅÆÇÈÉÊËÌÍÎÏÐÑÒÓÔÕÖ×ØÙÚÛÜÝÞßàáâãäåæçèéêëìíîïðñòóôõö÷øùúûüýþÿЉЊЋЌЍЎЏАБВГДЕЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯабвгде"

    filler = list(filler)
    filler.reverse()


    plot = []
    plot.extend(string.digits)
    plot.extend(string.ascii_uppercase)
    plot.extend(list("." * (255 - len(plot))))
    plot.append(" ")
    fails = 0
    for byte in nametable:
        if plot[byte] == ".":
            try:
                plot[byte] = filler.pop()
            except:
                fails += 1
    print(fails)

    print(len([p for p in plot if p != "."]))

    for i in range(0,256,16):
        print(''.join(plot[i:i+16]))
    return plot

In [7]:



def print_nametable_and_get_reverse_index(filler, nametable):
    # index = re.findall(r'[^\n]', filler)
    reverse_index = {c:i for  i, c in enumerate(filler)}


    for y,row in enumerate(nametable):
        for x,column in enumerate(row):
            print(filler[column],end="")
        print("")
    return reverse_index


In [8]:



def convert_attr_table_to_bytes(attr_table):
    attr_values = re.findall('[0123]', attr_table)
    attrs = []
    for row in range(0, len(attr_values), 32):
        data = attr_values[row:row+32]
        line1 = data[:16]
        line2 = data[16:]
        for i in range(0,16,2):
            tl = int(line1[i])
            tr = int(line1[i+1]) << 2
            bl = int(line2[i]) << 4
            br = int(line2[i+1]) << 6
            byte = br | bl | tr | tl
            attrs.append(byte)
    return attrs

In [9]:
def rebuild_nametable(table_grid, reverse_index, attr_table, target_file):
    attrs = convert_attr_table_to_bytes(attr_table)
    table_grid = re.findall(r'[^\n]', table_grid)

    nametable_rebuilt = [reverse_index[t] for t in table_grid]
    nametable_rebuilt.extend(attrs)
    # nametable_rebuilt = np.array(nametable_rebuilt, dtype=np.uint8)

    raw_to_nt(nametable_rebuilt, target_file)


In [10]:
game_array, game_grid, game_footer = get_array_grid_and_footer(OG_GAME_NAMETABLE)

In [13]:
level_array, level_grid, level_footer = get_array_grid_and_footer(OG_LEVEL_NAMETABLE)

In [12]:

title_array, title_grid, title_footer = get_array_grid_and_footer(OG_TITLE_NAMETABLE)

In [24]:
title_filler = print_and_get_filler(title_array)

0
192
0123456789ABCDEF
GHIJKLMNOPQRSTUV
WXYZ............
................
qksr............
.....àáâãäå.¼Ê..
lmd.æçèéêëìíÄÐ..
ijepîïðñòóôõÈÉÅÑ
fchaö÷øùúûüýÎÏÕÖ
bg.ßþÿЉЊЋЌЍЎ>ÔÝÞ
no.ËЏАБВГДЕЖÀÛÜº
*+.ÃЗИЙКЛМНО#.()
$%|tuvwxyz!&'Í^_
?@¿,-/:;.<=.ÒÓ.½
~¹Ç`¾×.{[]}ØÙÚ..
ÁÂÌ.ÆПРСТУФХ.». 


In [14]:
level_filler = print_and_get_filler(level_array)

0
86
0123456789ABCDEF
GHIJKLMNOPQRSTUV
WXYZu...)*,-/>;@
........qrstvwxy
[pm^z!#$%&'(:<=.
+?..............
abc.............
ijl]............
gkdf............
eh..............
on..............
................
................
................
................
............... 


In [19]:

game_filler = print_and_get_filler(game_array)

0
116
0123456789ABCDEF
GHIJKLMNOPQRSTUV
WXYZv...........
!#$%&¾¿Àrstuwxyz
:;<ÁÂÃ{|.@[]ÇÈÉ.
=>?ÄÅÆ}~¹^_`ÊËÌ/
º»¼½ÍÎÏb.'()*+,-
kldjÐg.cfea.....
opnhmq.i........
................
................
................
................
................
................
............... 


In [ ]:
print_attr_table(title_footer)

Original Attr Table:
2222222222222222
2222222222222222
2222222222222222
2333333333333332
2111111111111122
2222222222222222
2200000000000022
2000000000000022
2000000000333322
2200000000333322
2200000000333322
2200000000333322
2220311100000022
2222222222222222
2222222222222222
0000000000000000


In [15]:
print_attr_table(level_footer)

Original Attr Table:
2222222222222222
2222222222222222
2211122222211122
2211222212222222
2211222202222222
2213333333333122
2213333333333122
2213333333333122
2200000000000022
2200000000000022
2200000000000022
2200000000000022
2200000000000022
2222222222222222
2222222222222222
0000000000000000


In [ ]:
print_attr_table(game_footer)

Original Attr Table:
3333333333333333
3333333333333333
3333333333333333
3333332222233333
3333332222233333
3220032222233333
3220032222233333
3220032222233333
3220032222233333
3220032222233333
3220032222233333
3220032222233333
3220032222233333
3333333333333333
3333333333333333
0000000000000000


In [16]:
level_reverse_index = print_nametable_and_get_reverse_index(level_filler, level_grid)

ababcdcefcccdghghijcdgkkfefccgkh
ijijlgmlenlokfofofgpfekfgnenokfd
gkkfofdddgnqrrrrrrrrsdgkkfdgkhab
ghcqrrrrrrrt BuTYPE vrrrrrrrsdij
cllt       wxxxxxxxxy       vekf
ldlt                        vdgh
ofdt                        vabl
ekft    z!!!!!#    z!!!!!!# vijd
dght    $LEVEL%    $HEIGHT% vgkh
ablt    &'''''(    &''''''( vccd
ijdt  )*+*+*+*+*,  )*+*+*,  vloh
gkht  -0-1-2-3-4/  -0-1-2/  vlcd
ccdt  :;<;<;<;<;=  :;<;<;=  vdlg
loht  -5-6-7-8-9/  -3-4-5/  vcof
lcdt  >;?;?;?;?;@  >;?;?;@  vokf
dlgt                        vekf
coft                        vdgh
okft )********************, vabl
cght -    NAME  SCORE  LV / vijd
mclt :;;;;;;;;;;;;;;;;;;;;= vgkh
dldt - 1                  / vccd
gnct -                    / vloh
ablt - 2                  / vlcd
ijlt -                    / vdlg
ccdt - 3                  / vcof
l[]t >;;;;;;;;;;;;;;;;;;;;@ vokf
ldct                        vcab
denwxxxxxxxxxxxxxxxxxxxxxxxxylij
cdefcgkhgkkfefefabekfcg^fghghofc
lgngpfcdg^fgnclcijdgkncdabofofgp


In [20]:


game_reverse_index = print_nametable_and_get_reverse_index(game_filler, game_grid)


abccdeafgabcfhfhcibfjiklbifgadab
dhicibfjidhifeeaifmanaopneaifhfq
idaifmanaibrsssssssssstrssssssti
bcrsssssstcu LINESv   wu      wb
ciu vTYPEwcxyyyyyyyyyyzuTOP   wc
oaxyyyyyyzi!##########$u000000wc
feeabklbfej%          &u      wi
eafehopneai%          &uSCORE wb
jrsssssssst%          &u000000wn
cu'()*+,-/w%          &u      wb
iu        w%          &xyyyyyyzc
au :;<    w%          &feeafejfh
au =>?000 w%          &!####$ibf
au @[]    w%          &%NEXT&dhf
bu ^_`000 w%          &%    &ida
cu {|     w%          &%    &fhk
cu }~¹000 w%          &%    &dao
iu º»     w%          &%    &cfj
lu ¼½ 000 w%          &¾¿¿¿¿Àibc
pu ÁÂÃ    w%          &rssssstci
au ÄÅÆ000 w%          &uLEVELwcf
au ÇÈÉ    w%          &u     wib
bu ÊËÌ000 w%          &xyyyyyzfq
hu        w%          &bfgabdeai
ju ÍÎÏ000 w%          &Ðaifqibfe
cu        w%          &ifejidhda
ixyyyyyyyyz¾¿¿¿¿¿¿¿¿¿¿Àbklibifhb
bbfgadeabfjfjbdadeafeeacopfqbfeh
cnjibibdhbnacccbibkldaboafjicfee
oaifmacifmabicicdhopcbneabcfhdab


In [26]:

title_reverse_index = print_nametable_and_get_reverse_index(title_filler, title_grid)

ababcadefcghbaijdfkalmddfgdnafcg
fodhfcohlmhfobpbofcgijeqaeefccah
fcofrabaijfcgedhdlmhdfohdhna lmd
bcadhfo fccahhefsijbo  fka   ijq
hdfka        fo h  h          dh
bo                            ng
h  tuvwxvyz!tuvwxv#$%v &'()*+ dh
bp ,-/:;/ <=,-/:;/>?@/ []^_   ed
e  ` / {/ |}` / {/>~¹/ º»¼½   ee
h    /  /¾¿   /  /ÀÁÂ/ Ã»ÄÅ  dhe
bca  /  /ÆÇ   /  /È  / ÉÊ»Ë  ngh
hd   /  / ÌÍ  /  /ÎÏ / ÎÐ»Ñ   hd
fra  /  / ÒÓ  /  />Ô / >ÔÕÖ   dq
ah   ×  ×ØÙÚ  ×  ×ÀÛÜ× ÀÛÝÞ   eh
lm                            na
ij                           fcg
bp                      ß   frah
ed                   àáâãäå  hfg
hng                 æçèéêëìí  de
cdh                 îïðñòóôõ boh
dqa    PUSH START   ö÷øùúûüý hbp
eh                  þÿЉЊЋЌЍЎ  ef
na                  ЏАБВГДЕЖ  hd
cca                 ЗИЙКЛМНО  bo
md                            hd
je fra   П 1989  РСТУФХ      fco
ge  h                     d   lm
hhd   fg    dfcg bpbad ddfsbcaij
lmqa ddnafraefghdefofsbonghhdfgf
ijhfconcadhfodefshbcahhlmhdfkana


In [28]:
attr_table = """
3333333333333333
3333333333333333
3333333333333333
3333332222233333
3333332222233333
3220032222233333
3220032222233333
3220032222233333
3220032222233333
3220032222233333
3220032222233333
3220032222233333
3220032222233333
3333333333333333
3333333333333333
0000000000000000
"""



table_grid_raw = r"""
abccdeafgabcfhfhcibfjiklbifgadab
dhicibfjidhifeeaifmanaopneaifhfq
idaifmanaibrsssssssssstrssssssti
bcrsssssstcu LINESv   wu      wb
ciu vTYPEwcxyyyyyyyyyyzuTOP   wc
oaxyyyyyyzi!##########$u000000wc
feeabklbfej%          &u      wi
eafehopneai%          &uSCORE wb
jrsssssssst%          &u000000wn
cu'()*+,-/w%          &u      wb
iu        w%          &xyyyyyyzc
au :;<    w%          &feeafejfh
au =>?000 w%          &!####$ibf
au @[]    w%          &%NEXT&dhf
bu ^_`000 w%          &%    &ida
cu {|     w%          &%    &fhk
cu }~¹000 w%          &%    &dao
iu º»     w%          &%    &cfj
lu ¼½ 000 w%          &¾¿¿¿¿Àibc
pu ÁÂÃ    w%          &rssssstci
au ÄÅÆ000 w%          &uLEVELwcf
au ÇÈÉ    w%          &u     wib
bu ÊËÌ000 w%          &xyyyyyzfq
hu        w%          &bfgabdeai
ju ÍÎÏ000 w%          &Ðaifqibfe
cu        w%          &ifejidhda
ixyyyyyyyyz¾¿¿¿¿¿¿¿¿¿¿Àbklibifhb
bbfgadeabfjfjbdadeafeeacopfqbfeh
cnjibibdhbnacccbibkldaboafjicfee
oaifmacifmabicicdhopcbneabcfhdab
"""
                                
rebuild_nametable(table_grid_raw, game_reverse_index, attr_table, GAME_NAMETABLE)





In [52]:
level_attrs = """
2222222222222222
2222222222222222
2222222222222222
2222222222222222
2222222222222222
2222222222222222
2222222222222222
2222222222222222
2200000000000022
2200000000000022
2200000000000022
2200000000000022
2200000000000022
2222222222222222
2222222222222222
0000000000000000"""

level_raw = """
ababcdcefcccdghghijcdgkkfefccgkh
ijijlgmlenlokfofofgpfekfgnenokfd
gkkfofdddgngkkfggkkfgdgkkfdgkhab
ghcqrrrrrrrrrrrrrrrrrrrrrrrrsdij
cllt                        vekf
ldlt  TYPE              A   vdgh
ofdt                        vabl
ekft  MUSIC             0   vijd
dght                        vgkh
ablt  LEVEL             00  vccd
ijdt                        vloh
gkht  AB GARBAGE ROWS   00  vlcd
ccdt                        vdlg
loht  B LINES           25  vcof
lcdt                        vokf
dlgt  REUSE GARBAGE     0   vekf
coft                        vdgh
okft )********************, vabl
cght -    NAME  SCORE  LV / vijd
mclt :;;;;;;;;;;;;;;;;;;;;= vgkh
dldt - 1                  / vccd
gnct -                    / vloh
ablt - 2                  / vlcd
ijlt -                    / vdlg
ccdt - 3                  / vcof
l[]t >;;;;;;;;;;;;;;;;;;;;@ vokf
ldct                        vcab
denwxxxxxxxxxxxxxxxxxxxxxxxxylij
cdefcgkhgkkfefefabekfcg^fghghofc
lgngpfcdg^fgnclcijdgkncdabofofgp
"""

rebuild_nametable(level_raw, level_reverse_index, level_attrs, LEVEL_NAMETABLE)

In [51]:
title_attrs = """2222222222222222
2222222222222222
2222222222222222
2333333333333332
2111111111111122
2222222222222222
2200000000000022
2000000000000022
2000000000333322
2200000000333322
2200000000333322
2200000000333322
2220311100000022
2222222222222222
2222222222222222
0000000000000000"""

title_raw = """
ababcadefcghbaijdfkalmddfgdnafcg
fodhfcohlmhfobpbofcgijeqaeefccah
fcofrabaijfcgedhdlmhdfohdhna lmd
bcadhfo fccahhefsijbo  fka   ijq
hdfka        fo h  h          dh
bo                            ng
h  tuvwxvyz!tuvwxv#$%v &'()*+ dh
bp ,-/:;/ <=,-/:;/>?@/ []^_   ed
e  ` / {/ |}` / {/>~¹/ º»¼½   ee
h    /  /¾¿   /  /ÀÁÂ/ Ã»ÄÅ  dhe
bca  /  /ÆÇ   /  /È  / ÉÊ»Ë  ngh
hd   /  / ÌÍ  /  /ÎÏ / ÎÐ»Ñ   hd
fra  /  / ÒÓ  /  />Ô / >ÔÕÖ   dq
ah   ×  ×ØÙÚ  ×  ×ÀÛÜ× ÀÛÝÞ   eh
lm                            na
ij     OPTIONS FOR B GAME    fcg
bp                      ß   frah
ed                   àáâãäå  hfg
hng                 æçèéêëìí  de
cdh                 îïðñòóôõ boh
dqa      PUSH       ö÷øùúûüý hbp
eh      BUTTON      þÿЉЊЋЌЍЎ  ef
na                  ЏАБВГДЕЖ  hd
cca                 ЗИЙКЛМНО  bo
md                            hd
je fra   П 1989  РСТУФХ      fco
ge  h                     d   lm
hhd   fg    dfcg bpbad ddfsbcaij
lmqa ddnafraefghdefofsbonghhdfgf
ijhfconcadhfodefshbcahhlmhdfkana
"""

rebuild_nametable(title_raw, title_reverse_index, title_attrs, TITLE_NAMETABLE)